In [66]:
#!pip uninstall pytorch_lightning -y
!pip install lightning --upgrade
!pip install torchvision
!pip install pandas
!pip install scikit-learn
!pip install filetype

Requirement already up-to-date: lightning in /home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages (1.8.4.post0)


In [67]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
from torchvision import models
import pandas as pd
import numpy as np
import os

import lightning as pl

from torchvision.transforms import ToTensor

In [48]:
classes_dir = os.listdir("/media/amor/b30209c0-758e-49ec-ae71-e14cf76352f2/home/amor/db/")

#classes_dir

In [68]:
from sklearn import model_selection

from torch.utils.data import Dataset, DataLoader
from PIL import Image
import filetype


def filter_non_images(image_list):
    return list(filter(filetype.is_image, image_list))


class NSFWDataset(Dataset):
    """ Cassava Dataset """
    
    def __init__(self, root_dir="/media/amor/b30209c0-758e-49ec-ae71-e14cf76352f2/home/amor/db/", 
                 transform=transforms.Compose([
                    transforms.RandomPerspective(fill=0, p=1, distortion_scale=0.3),
                    transforms.RandomResizedCrop(224, ratio=(0.9, 1.1)),
                    ToTensor()]), 
                 stage=None):
        classes_dir = os.listdir(root_dir)
        self.class_to_index = dict()
        self.image_locations = list()
        self.labels = list()
        for i, class_loc in enumerate(classes_dir):
            class_path = os.path.join(root_dir, class_loc)
            image_paths = filter_non_images(
                [os.path.join(class_path, x) 
                 for x in os.listdir(class_path)]
            )
            self.image_locations.extend(image_paths)
            self.labels.extend([i for _ in range(len(image_paths))])
            self.class_to_index[class_loc] = i
            
        train_images, test_images, train_labels, test_labels = \
            model_selection.train_test_split(self.image_locations, self.labels,
                                             shuffle=True, random_state=4)
            
        if stage is not None and stage.lower() == "train":
            self.image_locations = train_images
            self.labels = train_labels
        elif stage is not None and stage.lower() == "test":
            self.image_locations = test_images
            self.labels = test_labels
            
        self.transform = transform
        
    def __len__(self):
        return len(self.image_locations)
    
    def __getitem__(self, idx):
        # Get and load image
        image_path = self.image_locations[idx]
        image = Image.open(image_path).convert("RGB")
        # Perform transforms if any
        if self.transform:
            image = self.transform(image)
        # Get label
        label = self.labels[idx]
        return image, label



In [69]:
dts = NSFWDataset()

dts.__getitem__(100000), torch.mean(dts.__getitem__(100000)[0]), len(dts.class_to_index)

KeyboardInterrupt: 

In [72]:
from torchvision import datasets

d = datasets.ImageFolder(root="/media/amor/b30209c0-758e-49ec-ae71-e14cf76352f2/home/amor/db/", 
                     transform=transforms.Compose([
                        transforms.RandomPerspective(fill=0, p=1, distortion_scale=0.3),
                        transforms.RandomResizedCrop(224, ratio=(0.9, 1.1)),
                        ToTensor()]), 
                     is_valid_file=filetype.is_image)


In [78]:
import json
json.dump(d.samples, open("valid_img_samples.json", "w"))
json.dump(d.class_to_idx, open("class_to_idx.json", "w"))

In [79]:
from torchvision import models

class ModelTrainer(pl.LightningModule):
    def __init__(self, n_classes=67):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = torch.nn.Linear(512, n_classes)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch

        y_hat = self.model(x)

        loss = F.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

In [81]:
dir(d)

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_format_transform_repr',
 '_is_protocol',
 '_repr_indent',
 'class_to_idx',
 'classes',
 'extensions',
 'extra_repr',
 'find_classes',
 'imgs',
 'loader',
 'make_dataset',
 'root',
 'samples',
 'target_transform',
 'targets',
 'transform',
 'transforms']

In [86]:
nsfw_model = ModelTrainer(len(dts.class_to_index))

trainer = pl.Trainer(accelerator="gpu")
trainer.fit(model=nsfw_model, 
            train_dataloaders=DataLoader(d, batch_size=64, shuffle=True, 
                                         num_workers=8, pin_memory=True ),
           
           )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Missing logger folder: /home/amor/Documents/code_dw/neural-styles/notebooks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total par

Training: 0it [00:00, ?it/s]

OSError: Caught OSError in DataLoader worker process 5.
Original Traceback (most recent call last):
  File "/home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 229, in __getitem__
    sample = self.loader(path)
  File "/home/amor/Documents/code_dw/neural-styles/venv39/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 268, in 

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  